In [1]:
# Sample Data Run
from path import Path
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import sqlalchemy as db
from sklearn import preprocessing

In [2]:
from config import password
# Set up variables to connect to SQL
database = "presidential_migration_db"
user = "postgres_blue"
password = f'{password}'
host = "presidential-migration-db.cexjpewgnljl.us-east-2.rds.amazonaws.com"
port = "5432"


In [3]:
# Create connection to Postgres DB using SQL Alchemy
engine = db.create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
engine.connect()

In [4]:
# Pull in Election Demographics table from Postgres
df = pd.read_sql_query('SELECT * from election_dem', con=engine)
df.sort_values(by=['STATE', 'YEAR'], inplace=True)
df.set_index(['STATE', 'YEAR'], inplace=True)
df.drop(columns=['previous_senate_winning_party', 'electoral_votes'], inplace=True)
df = df.rename(columns=str.upper)
df

OUTCOME  PCT_WHITE  PCT_BLACK  PCT_HISPANIC  PCT_OVER_60  \
STATE   YEAR                                                             
ALABAMA 2000        1   0.718555   0.260610      0.017049     0.173121   
        2004        1   0.712418   0.261851      0.025003     0.178516   
        2008        1   0.706231   0.263154      0.034697     0.188939   
        2012        1   0.700032   0.264896      0.039591     0.204672   
        2016        1   0.693757   0.267632      0.042229     0.224124   
...               ...        ...        ...           ...          ...   
WYOMING 2004        1   0.945082   0.008898      0.072678     0.166543   
        2008        1   0.940193   0.008999      0.084149     0.175860   
        2012        1   0.932112   0.013311      0.093638     0.193519   
        2016        1   0.928896   0.012540      0.098770     0.218721   
        2020        1   0.925033   0.012902      0.101267     0.241318   

              PCT_UNDER_40  REGION_MIDWEST  REGION_NORTHEAST  REGION_SOUTH  \
STATE   YEAR                                                                 
ALABAMA 2000      0.563600               0                 0             1   
        2004      0.543161               0                 0             1   
        2008      0.533678               0                 0             1   
        2012      0.521450               0                 0             1   
        2016      0.513533               0                 0             1   
...                    ...             ...               ...           ...   
WYOMING 2004      0.534961               0                 0             0   
        2008      0.537181               0                 0             0   
        2012      0.536336               0                 0             0   
        2016      0.535738               0                 0             0   
        2020      0.522559               0                 0             0   

              REGION_WEST  PAST_5_ELECTION_VOTING  BACHELORS_PER_CAPITA  \
STATE   YEAR                                                              
ALABAMA 2000            0                     1.0              0.064943   
        2004            0                     1.0              0.063746   
        2008            0                     1.0              0.062937   
        2012            0                     1.0              0.066716   
        2016            0                     1.0              0.068854   
...                   ...                     ...                   ...   
WYOMING 2004            1                     1.0              0.058593   
        2008            1                     1.0              0.058283   
        2012            1                     1.0              0.064167   
        2016            1                     1.0              0.074305   
        2020            1                     1.0              0.074103   

              BACHELORS_DEGREE_TOT  MED_HOUSE_INC  VIOLENT_CRIME_RATE  \
STATE   YEAR                                                            
ALABAMA 2000                288815          49855               21620   
        2004                288815          51395               19324   
        2008                296950          31051               21110   
        2012                321275          47091               21693   
        2016                334875          36480               25878   
...                            ...            ...                 ...   
WYOMING 2004                 29830          32023                1163   
        2008                 31825          40238                1330   
        2012                 36980          61285                1161   
        2016                 43410          47390                1431   
        2020                 44925          65134                1258   

              GDP_PER_CAPITA  GUNS_PER_HOUSEHOLD    POPULATION  \
STATE   YEAR                                

In [5]:
# Scale data using MinMaxScaler

scaler = preprocessing.MinMaxScaler()
names = df.columns
index = df.index
d = scaler.fit_transform(df)

scaled_df = pd.DataFrame(d, columns=names, index=index)
scaled_df.reset_index(inplace=True)
scaled_df.head(20)

,STATE,YEAR,OUTCOME,PCT_WHITE,PCT_BLACK,PCT_HISPANIC,PCT_OVER_60,PCT_UNDER_40,REGION_MIDWEST,REGION_NORTHEAST,...,PAST_5_ELECTION_VOTING,BACHELORS_PER_CAPITA,BACHELORS_DEGREE_TOT,MED_HOUSE_INC,VIOLENT_CRIME_RATE,GDP_PER_CAPITA,GUNS_PER_HOUSEHOLD,POPULATION,UNEMPLOYMENT_RATE,PREVIOUS_SENATE_OUTCOME
0,ALABAMA,2000,1.0,0.645715,0.423755,0.021122,0.429324,0.509877,0.0,0.0,...,1.0,0.065446,0.060219,0.438072,0.100458,0.021937,0.754839,0.099456,0.233645,1.0
1,ALABAMA,2004,1.0,0.637163,0.425796,0.037494,0.455484,0.422240,0.0,0.0,...,1.0,0.060060,0.060219,0.460934,0.089525,0.051943,0.719355,0.101558,0.327103,1.0
2,ALABAMA,2008,1.0,0.628541,0.427940,0.057447,0.506023,0.381582,0.0,0.0,...,1.0,0.056422,0.062111,0.158920,0.098030,0.075129,0.698387,0.106274,0.345794,1.0
3,ALABAMA,2012,1.0,0.619903,0.430807,0.067521,0.582315,0.329151,0.0,0.0,...,1.0,0.073421,0.067767,0.397040,0.100806,0.087013,0.762903,0.108724,0.570093,1.0
4,ALABAMA,2016,1.0,0.611159,0.435310,0.072949,0.676639,0.295208,0.0,0.0,...,1.0,0.083043,0.070929,0.239515,0.120733,0.105319,0.738710,0.109930,0.355140,1.0
5,ALABAMA,2020,1.0,0.607685,0.435660,0.079760,0.748106,0.270685,0.0,0.0,...,1.0,0.093194,0.075502,0.458589,0.116772,0.122325,0.738710,0.113004,0.355140,1.0
6,ALASKA,2000,1.0,0.636703,0.055005,0.070905,0.000000,0.782547,0.0,0.0,...,1.0,0.098456,0.003601,0.454402,0.014433,0.109396,0.856452,0.003350,0.392523,1.0
7,ALASKA,2004,1.0,0.618474,0.054380,0.088679,0.076084,0.645802,0.0,0.0,...,1.0,0.082499,0.003601,0.545256,0.017314,0.166535,0.822581,0.004163,0.495327,1.0
8,ALASKA,2008,1.0,0.602351,0.052215,0.097578,0.166148,0.587660,0.0,0.0,...,1.0,0.096201,0.004538,0.168510,0.018818,0.314069,0.875806,0.004872,0.411215,1.0
9,ALASKA,2012,1.0,0.584617,0.054587,0.113286,0.271011,0.573121,0.0,0.0,...,1.0,0.115813,0.005996,0.443729,0.018518,0.310531,0.866129,0.005954,0.476636,1.0


In [6]:
# Create training set from data before 2020
training_df = scaled_df
training_df

,STATE,YEAR,OUTCOME,PCT_WHITE,PCT_BLACK,PCT_HISPANIC,PCT_OVER_60,PCT_UNDER_40,REGION_MIDWEST,REGION_NORTHEAST,...,PAST_5_ELECTION_VOTING,BACHELORS_PER_CAPITA,BACHELORS_DEGREE_TOT,MED_HOUSE_INC,VIOLENT_CRIME_RATE,GDP_PER_CAPITA,GUNS_PER_HOUSEHOLD,POPULATION,UNEMPLOYMENT_RATE,PREVIOUS_SENATE_OUTCOME
0,ALABAMA,2000,1.0,0.645715,0.423755,0.021122,0.429324,0.509877,0.0,0.0,...,1.0,0.065446,0.060219,0.438072,0.100458,0.021937,0.754839,0.099456,0.233645,1.0
1,ALABAMA,2004,1.0,0.637163,0.425796,0.037494,0.455484,0.422240,0.0,0.0,...,1.0,0.060060,0.060219,0.460934,0.089525,0.051943,0.719355,0.101558,0.327103,1.0
2,ALABAMA,2008,1.0,0.628541,0.427940,0.057447,0.506023,0.381582,0.0,0.0,...,1.0,0.056422,0.062111,0.158920,0.098030,0.075129,0.698387,0.106274,0.345794,1.0
3,ALABAMA,2012,1.0,0.619903,0.430807,0.067521,0.582315,0.329151,0.0,0.0,...,1.0,0.073421,0.067767,0.397040,0.100806,0.087013,0.762903,0.108724,0.570093,1.0
4,ALABAMA,2016,1.0,0.611159,0.435310,0.072949,0.676639,0.295208,0.0,0.0,...,1.0,0.083043,0.070929,0.239515,0.120733,0.105319,0.738710,0.109930,0.355140,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,WYOMING,2004,1.0,0.961389,0.009512,0.135625,0.397424,0.387084,0.0,0.0,...,1.0,0.036879,0.000000,0.173350,0.003048,0.125137,0.866129,0.000385,0.158879,1.0
302,WYOMING,2008,1.0,0.954577,0.009679,0.159235,0.442602,0.396602,0.0,0.0,...,1.0,0.035484,0.000464,0.295304,0.003843,0.304468,0.919355,0.001315,0.074766,1.0
303,WYOMING,2012,1.0,0.943316,0.016774,0.178766,0.528234,0.392978,0.0,0.0,...,1.0,0.061957,0.001663,0.607755,0.003038,0.242160,0.850000,0.002076,0.317757,1.0
304,WYOMING,2016,1.0,0.938834,0.015506,0.189330,0.650440,0.390413,0.0,0.0,...,1.0,0.107563,0.003158,0.401479,0.004324,0.211181,0.866129,0.002275,0.308411,1.0


In [7]:
data_2024 = Path('Resources/Projections_2024.csv')
df_2024 = pd.read_csv(data_2024)
df_2024.head()

,STATE,PCT_WHITE,PCT_BLACK,PCT_HISPANIC,PCT_OVER_60,REGION,PAST_5_ELECTION_VOTING,OUTCOME_2020,TOTAL_POPULATION,BACHELORS_DEGREE_TOT,BACHELORS_PER_CAPITA,GDP_MILLIONS,GDP_PER_CAPITA,UNEMPLOYMENT_RATE,PREVIOUS_SENATE_OUTCOME
0,Alabama,0.682357,0.270320,0.055573,0.250762,South,1.0,1,5.090318e+06,3.703560e+05,0.072757,2.512859e+05,49365.46611,6.499536,1.0
1,Alaska,0.636881,0.036990,0.082088,0.207810,West,1.0,1,7.915178e+05,7.027107e+04,0.088780,6.701707e+04,84669.06021,6.768884,1.0
2,Arizona,0.807592,0.057454,0.338804,0.253147,West,0.8,0,7.852021e+06,5.992128e+05,0.076313,3.990693e+05,50823.76464,7.557536,0.0
3,Arkansas,0.783003,0.156023,0.094060,0.244763,South,1.0,1,3.155426e+06,2.219665e+05,0.070344,1.461867e+05,46328.66558,5.490623,1.0
4,California,0.701734,0.062491,0.420171,0.218229,West,0.0,0,4.144501e+07,4.615368e+06,0.111361,3.354606e+06,80941.13328,8.063319,0.0


In [8]:
df_2024 = pd.get_dummies(data=df_2024, columns=['REGION'])
df_2024 = df_2024.rename(columns=str.upper)

df_2024.columns

Index(['STATE', 'PCT_WHITE', 'PCT_BLACK', 'PCT_HISPANIC', 'PCT_OVER_60',
       'PAST_5_ELECTION_VOTING', 'OUTCOME_2020', 'TOTAL_POPULATION',
       'BACHELORS_DEGREE_TOT', 'BACHELORS_PER_CAPITA', 'GDP_MILLIONS',
       'GDP_PER_CAPITA', 'UNEMPLOYMENT_RATE', 'PREVIOUS_SENATE_OUTCOME',
       'REGION_MIDWEST', 'REGION_NORTHEAST', 'REGION_SOUTH', 'REGION_WEST'],
      dtype='object')

In [9]:
df_2024 = df_2024[['STATE', 'PCT_WHITE', 'PCT_BLACK', 'PCT_HISPANIC', 'PCT_OVER_60','REGION_MIDWEST', 'REGION_NORTHEAST', 'REGION_SOUTH', 'REGION_WEST',
       'PAST_5_ELECTION_VOTING', 'OUTCOME_2020', 'TOTAL_POPULATION',
       'BACHELORS_DEGREE_TOT', 'BACHELORS_PER_CAPITA', 'GDP_MILLIONS',
       'GDP_PER_CAPITA', 'UNEMPLOYMENT_RATE', 'PREVIOUS_SENATE_OUTCOME',
       ]].set_index('STATE')

In [12]:
# Scale data using MinMaxScaler

scaler = preprocessing.MinMaxScaler()
names = df_2024.columns
index = df_2024.index
d_2024 = scaler.fit_transform(df_2024)

scaled_df2024 = pd.DataFrame(d_2024, columns=names, index=index)
scaled_df2024


,PCT_WHITE,PCT_BLACK,PCT_HISPANIC,PCT_OVER_60,REGION_MIDWEST,REGION_NORTHEAST,REGION_SOUTH,REGION_WEST,PAST_5_ELECTION_VOTING,OUTCOME_2020,TOTAL_POPULATION,BACHELORS_DEGREE_TOT,BACHELORS_PER_CAPITA,GDP_MILLIONS,GDP_PER_CAPITA,UNEMPLOYMENT_RATE,PREVIOUS_SENATE_OUTCOME
STATE,,,,,,,,,,,,,,,,,
Alabama,0.626645,0.655480,0.071549,0.561803,0.0,0.0,1.0,0.0,1.0,1.0,0.109314,0.069637,0.039322,0.064342,0.039866,0.498941,1.0
Alaska,0.559644,0.075048,0.125042,0.266694,0.0,0.0,0.0,1.0,1.0,1.0,0.003993,0.003869,0.103427,0.008784,0.238744,0.539189,1.0
Arizona,0.811155,0.125954,0.642946,0.578192,0.0,0.0,0.0,1.0,0.8,0.0,0.176975,0.119795,0.053550,0.108899,0.048081,0.657036,0.0
Arkansas,0.774928,0.371155,0.149194,0.520586,0.0,0.0,1.0,0.0,1.0,1.0,0.061909,0.037115,0.029670,0.032654,0.022758,0.348181,1.0
California,0.655193,0.138485,0.807097,0.338281,0.0,0.0,0.0,1.0,0.0,0.0,1.000000,1.000000,0.193770,1.000000,0.217743,0.732614,0.0
Colorado,0.885278,0.100932,0.431836,0.396882,0.0,0.0,0.0,1.0,0.2,0.0,0.134720,0.167445,0.281447,0.114599,0.146051,0.366350,0.0
Connecticut,0.769963,0.298708,0.339764,0.601852,0.0,1.0,0.0,0.0,0.0,0.0,0.074255,0.075022,0.180012,0.085117,0.254692,0.634163,0.0
Delaware,0.606165,0.585103,0.186012,0.759255,0.0,0.0,1.0,0.0,0.0,0.0,0.009871,0.007359,0.082580,0.013860,0.219749,0.503645,0.0
District of Columbia,0.356885,1.000000,0.205719,0.021515,0.0,0.0,1.0,0.0,0.0,0.0,0.002719,0.039176,1.000000,0.037585,1.000000,0.607963,0.5


In [13]:
# Create testing set from 2020 data
testing_df = scaled_df2024
testing_df.columns

Index(['PCT_WHITE', 'PCT_BLACK', 'PCT_HISPANIC', 'PCT_OVER_60',
       'REGION_MIDWEST', 'REGION_NORTHEAST', 'REGION_SOUTH', 'REGION_WEST',
       'PAST_5_ELECTION_VOTING', 'OUTCOME_2020', 'TOTAL_POPULATION',
       'BACHELORS_DEGREE_TOT', 'BACHELORS_PER_CAPITA', 'GDP_MILLIONS',
       'GDP_PER_CAPITA', 'UNEMPLOYMENT_RATE', 'PREVIOUS_SENATE_OUTCOME'],
      dtype='object')

In [15]:
# Split training/testing data
y_train = training_df.OUTCOME
y_test = testing_df.OUTCOME_2020
X_train = training_df.drop(columns=['YEAR','STATE','OUTCOME', 'GUNS_PER_HOUSEHOLD', 'MED_HOUSE_INC', 'VIOLENT_CRIME_RATE', 'POPULATION', 'PCT_UNDER_40', 'BACHELORS_DEGREE_TOT'])
X_test = testing_df.drop(columns=['OUTCOME_2020','TOTAL_POPULATION','BACHELORS_DEGREE_TOT','GDP_MILLIONS'])

In [16]:
X_train.columns

Index(['PCT_WHITE', 'PCT_BLACK', 'PCT_HISPANIC', 'PCT_OVER_60',
       'REGION_MIDWEST', 'REGION_NORTHEAST', 'REGION_SOUTH', 'REGION_WEST',
       'PAST_5_ELECTION_VOTING', 'BACHELORS_PER_CAPITA', 'GDP_PER_CAPITA',
       'UNEMPLOYMENT_RATE', 'PREVIOUS_SENATE_OUTCOME'],
      dtype='object')

In [17]:
X_test.columns

Index(['PCT_WHITE', 'PCT_BLACK', 'PCT_HISPANIC', 'PCT_OVER_60',
       'REGION_MIDWEST', 'REGION_NORTHEAST', 'REGION_SOUTH', 'REGION_WEST',
       'PAST_5_ELECTION_VOTING', 'BACHELORS_PER_CAPITA', 'GDP_PER_CAPITA',
       'UNEMPLOYMENT_RATE', 'PREVIOUS_SENATE_OUTCOME'],
      dtype='object')

In [18]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(solver='lbfgs',
                                random_state=1)

In [19]:
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [20]:
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"2024 Prediction": y_pred, "2020 Actual": y_test})
results

,2024 Prediction,2020 Actual
STATE,,
Alabama,1.0,1.0
Alaska,1.0,1.0
Arizona,0.0,0.0
Arkansas,1.0,1.0
California,0.0,0.0
Colorado,0.0,0.0
Connecticut,0.0,0.0
Delaware,0.0,0.0
District of Columbia,0.0,0.0


In [21]:
results.to_csv('2024_predictions.csv')